In [1]:
# std packages
%matplotlib inline
import numpy as np
import scipy.interpolate as interp
import scipy.integrate as integ
import scipy.signal as sig
import scipy.optimize as opt
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.gridspec as gridspec
import h5py as h5
import os, sys
import timeit

plt.rc('figure', figsize=(9, 7))
plt.rcParams.update({'text.usetex': True,
                     'text.latex.preamble':r'\usepackage{amsmath}',
                     'font.family': 'serif',
                     'font.serif': ['Georgia'],
                     'mathtext.fontset': 'cm',
                     'lines.linewidth': 2.5,
                     'font.size': 20,
                     'xtick.labelsize': 'large',
                     'ytick.labelsize': 'large',
                     'xtick.direction': 'in',
                     'ytick.direction': 'in',
                     'xtick.major.width': 1.7,
                     'ytick.major.width': 1.7,
                     'xtick.major.size': 7.,
                     'ytick.major.size': 7.,
                     'ytick.right':True, 
                     'axes.labelsize': 'large',
                     'axes.titlesize': 'large',
                     'axes.grid': True,
                     'grid.alpha': 0.5,
                     'lines.markersize': 12,
                     'legend.borderpad': 0.2,
                     'legend.fancybox': True,
                     'legend.fontsize': 17,
                     'legend.framealpha': 0.7,
                     'legend.handletextpad': 0.5,
                     'legend.labelspacing': 0.2,
                     'legend.loc': 'best',
                     'savefig.bbox': 'tight',
                     'savefig.pad_inches': 0.05,
                     'savefig.dpi': 80,
                     'pdf.compression': 9})


import lal
import lalsimulation as lals


# packages used to generate ODE-Phenom waveforms
from src.LAL_constants import *
from src import waveLib as wL
from src import misc

In [2]:
Mt_Ms = 30
t_Mt = G*Mt_Ms*Ms/c**3.


qq = 1./2
M1_Ms, M2_Ms = Mt_Ms/(1.+qq), qq*Mt_Ms/(1.+qq)

chi1p, chi1z = 0.6,  0.2
phi_1 = 0.5 * np.pi
chi1x = chi1p * np.cos(phi_1)
chi1y = chi1p * np.sin(phi_1)

chi2p, chi2z = 0.6,  0.2
phi_2 = 0. * np.pi
chi2x = chi2p * np.cos(phi_2)
chi2y = chi2p * np.sin(phi_2)

dist_Mpc = 1
phi_ref = .55* np.pi #-0.3*np.pi
iota = np.pi/3

f_ref = 50


f_lower = 10
delta_t, delta_f = misc.get_delta_t_delta_f(f_lower, M1_Ms*Ms, M2_Ms*Ms)

# aLIGO has tiny sensitivity beyond 2k Hz
if 0.5/delta_t > 2048:
    delta_t = 1/4096.
    
print('f_max [Hz]:',.5/delta_t)
print('1/Delta f ~ t_duration [s]:', 1./delta_f)


freq = np.arange(f_lower, .5/delta_t, delta_f)
print('number of freq bins:', len(freq))

f_max [Hz]: 2048.0
1/Delta f ~ t_duration [s]: 32.0
number of freq bins: 65216


In [4]:
# make sure the waveform runs

### IMRPhenomXODE ###

# modes in the co-precessing frame; 
# note only m<0 modes have f>0 support
ll_list_neg = np.array([ 2,  2,  3,  3,  4])
mm_list_neg = np.array([-2, -1, -3, -2, -4])

# define LAL like input
aux_par = lal.CreateDict()
kwargs_XODE = {
    'freqs':freq,
    'approximant': 'XODE',
    'll_list_neg':ll_list_neg,
    'mm_list_neg':mm_list_neg,
    'use_N4LO_prec':True,
    'SEOB_22_cal':True,
    'SEOB_HM_cal':True,
    'update_spin': False,
    'mass1':M1_Ms,
    'mass2':M2_Ms,
    'spin1x':chi1x, 
    'spin1y':chi1y,
    'spin1z':chi1z,
    'spin2x':chi2x, 
    'spin2y':chi2y,
    'spin2z':chi2z,
    'f_ref': f_ref,
    'phi_ref':phi_ref,
    'iota':iota,
    'f_lower': freq[0],
    'distance': dist_Mpc,    
    'aux_par':aux_par, 
    'atol':3e-4,
    'rtol':3e-4
}

# XODE
t_run0 = timeit.default_timer()
hp_XODE, hc_XODE = wL.get_hp_hc_f_sequence(**kwargs_XODE)
t_run1 = timeit.default_timer()

# it may take some time for compiling the code when running it the first time
# the subsequent evaluation should be efficient <~ 40 ms for the example
print('time (s) per waveform: ', t_run1 - t_run0)



### IMRPhenomXPHM ###
kwargs_XPHM = kwargs_XODE.copy()
# 102 for NNLO 223 for MSA
lals.SimInspiralWaveformParamsInsertPhenomXPrecVersion(kwargs_XPHM['aux_par'], 223)
kwargs_XPHM['approximant'] = 'XPHM'
hp_XPHM, hc_XPHM = wL.get_hp_hc_f_sequence(**kwargs_XPHM)

time (s) per waveform:  0.1505344270000002


## Check the performance using prun

### XODE

In [5]:
%%prun
for i in range(100):
    hp_XODE, hc_XODE = wL.get_hp_hc_f_sequence(**kwargs_XODE)

### XPHM

In [6]:
%%prun
for i in range(100):
    hp_XPHM, hc_XPHM = wL.get_hp_hc_f_sequence(**kwargs_XPHM)